# VinBigData Chest X-ray Abnormalities Detectionm

## Automatically localize and classify thoracic abnormalities from chest radiographsm

Page:
https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection

# 1. Import Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import pathlib
from pprint import pprint

#img
import cv2

#pytorch
import torch
from torch import nn
from torch import functional as F
from torch import optim
from torch.utils.data import Dataset,DataLoader,random_split
from torchvision import transforms
from torch.nn import Module
from torchvision import models
from PIL import Image
from albumentations.pytorch.transforms import ToTensorV2
from tqdm.notebook import tqdm
import albumentations as A

#FasterRCNN
#from torchvision.models.detection import FasterRCNN
#from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
#from torchvision.models.detection.rpn import AnchorGenerator

#dicom
import pydicom

#set Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Path

In [ ]:
#-----path-----
#train csv
train_csv_path=pathlib.\
        Path("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
sample_sub_path=pathlib.\
        Path("../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv")
#dicom data
train_data_path=pathlib.\
        Path("../input/vinbigdata-chest-xray-abnormalities-detection/train")
test_data_path=pathlib.\
        Path("../input/vinbigdata-chest-xray-abnormalities-detection/test")

#pathの確認
print(pathlib.Path.exists(train_csv_path),
      pathlib.Path.exists(train_data_path),
      pathlib.Path.exists(test_data_path)
     )

In [ ]:
#get train image ids
image_ids=[x for x in train_data_path.iterdir() if x.is_file()]
len(image_ids)


In [ ]:
#train csv
df=pd.read_csv(train_csv_path)
df.head()

In [ ]:
#image id
sample_=pathlib.\
    Path('../input/vinbigdata-chest-xray-abnormalities-detection/train/000434271f63a053c4128a0ba6352c7f.dicom')

print("suffix",image_ids[0].suffix)
print("non suffix:",image_ids[0].stem)

#test sample
df[df["image_id"]==sample_.stem]

In [ ]:
#fill NaN
print(df.isnull().sum())
df.fillna(0,inplace=True)

In [ ]:
#classid
print(df["class_id"].unique())
print("uniques:",len(df["class_id"].unique()))

## 5.2 Normalize Dicom Image

### idea is below

[reference] https://www.kaggle.com/raddar/popular-x-ray-image-normalization-techniques

### [1] No-Normalization

### [2] Histogram normalization
The general idea is to make pixel distribution uniform. This makes X-rays appear a little darker. This generates view, which radiologist would not see in his standard workplace.\ Such normalization is used in popular open-source X-ray datasets, such as CheXpert.

img = read_xray(str(sample_ids))\
img = exposure.equalize_hist(img)


### [3] CLAHE normalization
This method produces sharper images and is quite often used in chest X-ray research. This generates view, which radiologist would not see in his standard workplace. However, it closely resembles the "bone-enhanced" view in some X-rays done (usually due to broken ribs).

img = read_xray(str(sample_ids))\
img = exposure.equalize_adapthist(img/np.max(img))

## My issue 1:

## is this error?

## Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' valuef"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "

In [ ]:
def read_xray_normalized(image_ids):
    #read dicom data
    ds=pydicom.dcmread(image_ids)
    #->to ndarray
    dcm_arr=ds.pixel_array
    
    #Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). 
    #It's recommended that you change the 'Bits Stored' value
    #f"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "
    
    #is this need...?
    dcm_arr = np.right_shift(dcm_arr, ds.BitsAllocated - ds.BitsStored)
    
    #normalize arr
    amin=np.amin(dcm_arr)
    amax=np.amax(dcm_arr)
    scale = 255.0/(amax-amin) # set scale
    arr_rescaled = dcm_arr*scale # >rescale 0-255
    arr_normalized = np.uint8(arr_rescaled) #->uint8
    return arr_normalized

In [ ]:
temp_img=read_xray_normalized(image_ids[5])
print(temp_img.shape)
plt.figure(figsize=(8,8))
plt.imshow(temp_img,cmap="bone")
plt.show()

In [ ]:
#Histogram normalization(type:ndarray)
from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage import exposure

image = exposure.equalize_hist(temp_img)
plt.figure(figsize=(8,8))
plt.imshow(image,cmap="bone")
plt.show()

## simple Data Augmentation

In [ ]:
#Set Image Augumentation
from torchvision import transforms
import albumentations

#transforms.Grayscale(3) like RGB channels

transform=transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(3),
        transforms.ToTensor(),
        ]) 

### Resize Image and BoundingBox

In [ ]:
def resize(image, boxes, width, height):
    # 現在の高さと幅を取得しておく
    c_height, c_width = image.shape[:2]
    img = cv2.resize(image, (width, height))
    
    # 圧縮する比率(rate)を計算
    r_width = width / c_width
    r_height = width / c_height
    
    # 比率を使ってBoundingBoxの座標を修正
    new_boxes = []
    for box in boxes:
        x,y,w,h=box
        x = int(x * r_width)
        y = int(y * r_height)
        w = int(w * r_width)
        h = int(h * r_height)
        new_box =[x, y, w, h]
        new_boxes.append(new_box)
    return img, new_boxes

### Dataset and DataLoader

In [ ]:
class My_Dataset(Dataset):
    def __init__(self,df,):
        
        #dataframeを格納する
        self.df = df
        self.image_ids=df["image_id"].unique()
        self.image_dir=pathlib.\
                    Path("../input/vinbigdata-chest-xray-abnormalities-detection/train")
        #columnsを設定する
        self.box_col=["y_min","y_min","x_max","y_max"]
        #transform
        self.transform=transforms.Compose(
            [
            transforms.ToPILImage(),
            transforms.Grayscale(3),
            transforms.ToTensor(),
            ]) 

    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self,index,transform=False):
        
        #get dicom_arr
        image_id=self.image_ids[index]
        image=read_xray_normalized(str(self.image_dir/image_id)+".dicom")
        
        
        #target
        records = self.df[(self.df['image_id'] == image_id)]
        records = records.reset_index(drop=True)
        
        if records.loc[0, "class_id"] == 0:
            records = records.loc[[0], :]
        
        #-----bounding box-----
        boxes = records[self.box_col].values.astype(np.float32)
        #----area-----
        #bbox:[x,y,w,h] and area=(w-x)*(h-y)
        area = (boxes[:,2] - boxes[:,0]) * (boxes[:,3] - boxes[:,1])
        area = area.astype(np.float32)
        
        #----labels-----
        labels = torch.tensor(records["class_id"].values, dtype=torch.int64)
        
        # suppose all instances are not crowd
        #iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
                
        #-----[target]:dict-----
        target = {}
        target['boxes'] = torch.tensor(boxes)
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        #target['area'] = torch.tensor(area)
        #target['iscrowd'] = iscrowd
        target["image_row_shape"]=torch.tensor(image.shape)
        target["dicom_id"]=image_id
        
        #Transoformed Image
        #transform
        #image_transformed=self.transform(image.astype(np.float32))
        
        
        #resize image and bbox
        #resize scale;width,height=[512,512]
        width=512
        height=512
        image_resized,boxes_resized=resize(image,boxes,width, height)
        #print("boxes_resized:",boxes_resized)
        target["boxes_resized"]=torch.tensor(boxes_resized)
        image_transformed=self.transform(image_resized.astype(np.float32))
        
        return image_transformed, target

In [ ]:
#change collate_fn
def collate_fn(batch):
    imgs, targets= list(zip(*batch))
    
    imgs = torch.stack(imgs)
    targets = list(targets)
    
    return imgs,targets

In [ ]:
train_dataset=My_Dataset(df=df)
train_dataloader=DataLoader(train_dataset,
                            batch_size=2,shuffle=True, 
                            collate_fn= collate_fn)

# Sample Output Test
image,target =next(iter(train_dataloader))
print("------image-----")
print("image_tensor:",image.shape)
print("-----target-----")
print(target[0])
print(target[1])

## EfficientDet

[reference]\
1.github : https://github.com/rwightman/efficientdet-pytorch

2.kaggle : https://www.kaggle.com/shonenkov/training-efficientdet

### Import Packages for EfficientDet-pytorch

In [ ]:
!pip install timm
!pip install omegaconf
!pip install pycocotools
!pip install effdet


# Using EfficientDet d1 sample

In [ ]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain,DetBenchPredict
from effdet.efficientdet import HeadNet


config = get_efficientdet_config('tf_efficientdet_d1')
config.image_size = [512,512]
config.norm_kwargs=dict(eps=.001, momentum=.01)

#BackBone=True
net = EfficientDet(config, pretrained_backbone=True)

#default 90 -> 19
net.reset_head(num_classes=15)
net.class_net = HeadNet(config, num_outputs=config.num_classes)


#[MODE]:Train
net=DetBenchTrain(net, config)
net.train()
print("Loaded pretrained weights")

In [ ]:
#configを確認
print(net.config)

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# sample output


In [ ]:
summary_loss = AverageMeter()

net.train()
for i in range(1):
    images,targets=next(iter(train_dataloader))
    #print(images.shape)
    #print(targets)
    
    batch_size = images.shape[0]
    
    boxes = [target['boxes_resized'].to(device).float() for target in targets]
    labels = [target['labels'].to(device).float() for target in targets]
    print("images:",images.shape)
    print("boxes:",boxes)
    print("labels:",labels)
    
    
    #train
    target_dict={}
    target_dict["bbox"]=boxes
    target_dict["cls"]=labels
    output=net(images,target_dict)
    loss=output["loss"]
    print("loss:",loss)
    
    #loss.backward()
    summary_loss.update(loss.detach().item(), batch_size)

print(summary_loss.avg)

# set train models 

In [ ]:
from torch import Tensor as tensor

def train_loss(images,targets):
   
    batch_size = images.shape[0]
    
    boxes = [target['boxes_resized'].to(device).float() for target in targets]
    labels = [target['labels'].to(device).float() for target in targets]
    
    img_scale = torch.Tensor([1.0] * batch_size)
    img_size = torch.Tensor([images[0].shape[-2:]] * batch_size,)
    
    target_dict={}
    target_dict["bbox"]=boxes
    target_dict["cls"]=labels
    #target_dict["img_scale"]=img_scale
    #target_dict["img_size"]=img_size
    
    
    output=net(images,target_dict)
    train_loss=output["loss"]
    
    return train_loss

In [ ]:
def validation_loss(images,targets):
   
    batch_size = images.shape[0]
    
    boxes = [target['boxes_resized'].to(device).float() for target in targets]
    labels = [target['labels'].to(device).float() for target in targets]

    img_scale = torch.Tensor([1.0] * batch_size)
    img_size = torch.Tensor([images[0].shape[-2:]] * batch_size,)

    target_dict={}
    target_dict["bbox"]=boxes
    target_dict["cls"]=labels
    target_dict["img_scale"]=img_scale
    target_dict["img_size"]=img_size
    
    with torch.no_grad():
        output=net(images,target_dict)
        val_loss=output["loss"]
    
    return val_loss

In [ ]:
def Train_ObjDetect(net, dataloaders_dict, optimizer, num_epochs):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("device：", device)

    net.to(device)

    torch.backends.cudnn.benchmark = True
        
    best_summary_loss = 10**5
    
    #dict
    #hist={"loss":[],"val_loss":[],}
    
    # loop epoch
    for epoch in range(num_epochs):
        print('---Epoch {}/{}---'.format(epoch+1, num_epochs))

        for phase in ['train', 'val']:
            
            if phase == 'train':
                net.train()  
            else:
                net.eval()
            
            for images, targets in dataloaders_dict[phase]:
                
                optimizer.zero_grad()
                
                summary_loss = AverageMeter()
                
                if phase=="train":
                    loss=train_loss(images,targets)
                    print("train loss:",loss)
                    
                    batch_size=images.shape[0]
                    summary_loss.update(loss.detach().item(), batch_size)
                
                    loss.backward()
                    optimizer.step()
                    
                else:
                    loss=validation_loss(images,targets)
                    print("val loss:",loss)
                    batch_size=images.shape[0]
                    summary_loss.update(loss.detach().item(), batch_size)
                    
            if summary_loss.avg < best_summary_loss:
                best_summary_loss = summary_loss.avg
            #print(summary_loss.avg)
           
    return 

In [ ]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain,DetBenchPredict
from effdet.efficientdet import HeadNet

config = get_efficientdet_config('tf_efficientdet_d1')
config.image_size = [512,512]
config.norm_kwargs=dict(eps=.001, momentum=.01)

#BackBone=True
net = EfficientDet(config, pretrained_backbone=True)

#default 90 -> 19
net.reset_head(num_classes=15)
net.class_net = HeadNet(config, num_outputs=config.num_classes)


#[MODE]:Train
net=DetBenchTrain(net, config)
net.train()
print("Loaded pretrained weights")

In [ ]:
from sklearn.model_selection import train_test_split
test_size=0.2
df_train,df_val=train_test_split(df,test_size=test_size,
                                random_state=64)

#分割を確認
print(df_train.shape)
print(df_val.shape)

# TEST CODE　

In [ ]:
# test sample with reducing data
df_train=df_train[:30]
df_val=df_val[:10]

In [ ]:
#DataSet
train_dataset=My_Dataset(df_train)
val_dataset=My_Dataset(df_val)

#Dataloader
batch_size=5
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)
val_dataloader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False,collate_fn=collate_fn)

#dict
dataloaders_dict={"train":train_dataloader,"val":val_dataloader}


In [ ]:
learning_rate=1e-3

num_epochs=1
#-----optimizer-----
optimizer=torch.optim.AdamW(net.parameters(), lr=learning_rate)
#-----train model-----
from tqdm import tqdm

Train_ObjDetect(net=net,
            dataloaders_dict=dataloaders_dict,
            optimizer=optimizer,
            num_epochs=num_epochs)

## My issue 1:

## is this error? 

> ## Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' valuef"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "

### add  below code to solve
dcm_arr = np.right_shift(dcm_arr, ds.BitsAllocated - ds.BitsStored)

## but its causion continues...

# My Big Current issue 2

## get Loss output NaN... immediately

### to solve this problem

---Epoch 1/1---
train loss: tensor(37132.6797, grad_fn=<AddBackward0>)\
train loss: tensor(117652.2500, grad_fn=<AddBackward0>)\
train loss: tensor(nan, grad_fn=<AddBackward0>)
    

## To solve:
    
1. change leaning rate
    
   lr=1e-5 -> output Nan\
   lr1e-7 -> output nan...




## I'm beginner of Objective Detection...


I would be grateful if you could give me some adbice

# Next Inference test Data...